In [1]:
import biosteam as bst
import thermosteam as tmo
import biorefineries as bf
from biorefineries import cellulosic
from lignin_saf.rcf import * 
from lignin_saf.reactor import *

In [2]:
chems = cellulosic.create_cellulosic_ethanol_chemicals().copy()   # Creating a copy of the cellulosic ethanol chemicals

# Appending methanol and other relevant chemicals I would require 
add_chems = bst.Chemicals(['Methanol',  
                           'Hydrogen'])

chems.extend(add_chems)

bst.settings.set_thermo(chems)

In [3]:
bst.settings.CEPCI = 541.7

In [4]:
# Defining poplar group based off composition given in Bartling et al [1] Table S1

chems.define_group(
    name='Poplar',
    IDs=['Cellulose',   # Cellulase break this down to glucose
         'Xylan',       # Hemicellulose
         'Arabinan',    # Hemicellulose
         'Mannan',      # Hemicellulose
         'Galactan',    # Hemicellulose
         'Sucrose',   
         'Lignin',
         'Acetate',
         'Extract',
         'Ash'],
    composition=[0.464,      # Dry wt composition, feed also has 20 wt% moisture content
                 0.134,
                 0.002,
                 0.037,
                 0.014,
                 0.001,
                 0.285,
                 0.035,
                 0.016,
                 0.012],
    wt=True
)


In [5]:
# RCF conditions from Bartling et al 2021

rcf_temp = 200 + 273.15            # K 
rcf_pressure = 60 * 1e5            # Pascal
rcf_residence_time = 3             # hr
reactor_operation = 'Isothermal'
solvent_water_ratio = 9            # 9:1 methanol to water ratio on a vol basis
catalyst_biomass = 0.1             # catalyst to dry biomass bed ratio
replacement = 1                    # once replacement of catalyst every year
biomass_deliginfication = 0.7      # 70% biomass is assumed deligniified
methanol_decomp = 0.005              # methanol decompositin rate %
cellulose_retention = 0.9          # 90% 
xylose_retention = 0.93            # 93% 
methanol_to_biomass = 9           # 90 mL / g of biomass from https://doi.org/10.1016/j.copbio.2018.12.005



lignin_monomers = 0.5
lignin_dimers = 0.25
lignin_oligomers = 0.25 





In [7]:
poplar_in = bst.MultiStream('Poplar_In',
                s=[('Poplar', 2e6)], l=[('Water', 0.2*2e6)], 
                 phases = ('s','l'), units='kg/d')

In [9]:
meoh_in = bst.Stream('Methanol_In',
                     Methanol = methanol_to_biomass*poplar_in.F_mass, Water = methanol_to_biomass*poplar_in.F_mass*(1/solvent_water_ratio),
                     units = 'L/hr',
                     T = 300,
                     P = 1e5,
                     phase = 'l'
)

In [ ]:

import biosteam as bst, math, typing, numpy as np

from math import ceil
from typing import Optional
from biosteam.units.design_tools import (
    PressureVessel, size_batch
)
 
'''
__all__ = (
    'RCFReactor', 'RCF', 'ReductiveCatalyticFractionation'
)
'''
class RCFReactor(bst.Unit, bst.units.design_tools.PressureVessel):



    auxiliary_unit_names = (
        'pump_1', 'heat_exchanger_1'
    )

    _N_ins = 2
    _N_outs = 2
    
    _units = {**PressureVessel._units,
              'Pressure drop': 'bar',
              'Batch time': 'hr',
              'Loading time': 'hr',
              'Residence time': 'hr',
              'Total volume': 'm3',
              'Reactor volume': 'm3'}
    


    # Default operating temperature [K]
    T_default: float = 463.15  # 190 C from  https://doi.org/10.1016/j.joule.2017.10.004

    #: Default operating pressure [Pa]
    P_default:  float = 6e6 # 60 MPa from https://doi.org/10.1016/j.joule.2017.10.004
    
    #: Default residence time [hr]
    tau_default: float = 3

    #: Default fraction of working volume over total volume.
    V_wf_default: float = 0.8


    #: Default length to diameter ratio.
    length_to_diameter_default: float = 3
    
    #: Default cleaning and unloading time (hr).
    tau_0_default: float  = 1 # from https://doi.org/10.1039/D1EE01642C
    
    # Default superficial velocity of solvent (ft/s)
    superficial_velocity_default: float = 0.3 # Just assumed 

    # Default methanol decomposition (%)
    methanol_decomposition_default: float = 0.005 # from https://doi.org/10.1039/D1EE01642C

    # Default poplar bed void fraction 
    epsilon_default: float = 0.5


    def _init(
            self,
            T: Optional[float] = None, 
            P: Optional[float] = None,
            tau: Optional[float] = None,
            V_wf: Optional[float] = None,
            length_to_diameter: Optional[float] = None, 
            vessel_material: Optional[str] = None,
            vessel_type: Optional[str] = None,
            tau_0: Optional[float] = None,  
            superficial_velocity: Optional[float] = None,
            methanol_decomposition: Optional[float] = None,
            epsilon: Optional[float] = None,
            N_beds = 4, *,
            reaction
            ):
        
        
        self.T = self.T_default if T is None else T
        self.P = self.P_default if P is None else P
        self.tau = self.tau_default if tau is None else tau
        self.V_wf = self.V_wf_default if V_wf is None else V_wf
        self.length_to_diameter = self.length_to_diameter_default if length_to_diameter is None else length_to_diameter
        self.vessel_material = 'Stainless steel 316' if vessel_material is None else vessel_material
        self.vessel_type = 'Vertical' if vessel_type is None else vessel_type
        self.tau_0 = self.tau_0_default if tau_0 is None else tau_0
        self.superficial_velocity = self.superficial_velocity_default if superficial_velocity is None else superficial_velocity
        self.methanol_decomposition = self.methanol_decomposition_default if methanol_decomposition is None else methanol_decomposition
        self.epsilon = self.epsilon_default if epsilon is None else epsilon
        
        if N_beds != 4:
            raise ValueError('only 4 biomass beds are valid')
        self.reaction = reaction
        self.N_beds = N_beds
        pump_1 = self.auxiliary('pump_1', bst.Pump, ins = self.ins[1])
        heat_exchanger_1 = self.auxiliary('heat_exchanger_1', bst.HXutility, ins = pump_1.outs[0])


    def _size_bed(self):
        biomass, solvent = self.ins
        #used_biomass, used_solvent = self.outs
        #used_solvent.copy_like(solvent)
        #used_solvent.P = self.P
        #used_solvent.T = self.T

        biomass_rho = 150  # [kg/m3] density of poplar of particle size 0.4 x 0.4 cm from Table 2-2 of https://digital.lib.washington.edu/server/api/core/bitstreams/ac598b6f-f82a-423a-a8c9-2cd4ecd9aac1/content
        total_mass_flow = biomass.F_mass + solvent.F_mass
        solvent_mass_frac = solvent.F_mass/total_mass_flow
        biomass_mass_frac = biomass.F_mass/total_mass_flow
        mixture_avg_density = biomass_rho * biomass_mass_frac + solvent.rho * solvent_mass_frac 

        total_tau = self.tau + self.tau_0 

        total_volume = (total_mass_flow*total_tau)/mixture_avg_density

        volume_per_reactor = total_volume/self.N_beds

        actual_per_reactor_volume = volume_per_reactor/0.5  # frac_working volume

        Q = solvent.F_vol * (3.28**3) # ft3/hr
        u = self.superficial_velocity * 3600   # ft/hr
        self.area = area = Q / u
        self.diameter = diameter = 4 * (area/np.pi) ** 0.5

        self.length = bed_length = actual_per_reactor_volume/area


        #self.length = total_length = total_biomass_volume/area # Length calculated based off L/D
        #if self.N_beds == 4:  # always true
        #
        #     bed_length = total_length/4
        return diameter, bed_length, actual_per_reactor_volume
        
        




        
        
        
        biomass = self.ins[0]
        biomass_massflow = biomass.F_mass # kg/hr
        total_biomass_weight = biomass_massflow * (self.tau+self.tau_0)
        total_biomass_volume = (total_biomass_weight/self.rho) * 1.5 # 50% overdesign to accomodate solvent flow 

        solvent = self.ins[1]
        

    def _run(self):
        biomass, solvent = self.ins
        used_biomass, used_solvent = self.outs
        used_solvent.copy_like(solvent) 
        used_solvent.imass['Methanol'] = used_solvent.imass['Methanol']*(1-self.methanol_decomposition) # 0.5% methanol lost due to decomposition
        used_biomass.copy_like(biomass)
        used_solvent.P = self.P
        used_solvent.T = self.T

        self.reaction(used_biomass)
        

        solubilized_lignin = used_biomass.imass['SolubleLignin'] 
        used_solvent.imass['SolubleLignin'] += solubilized_lignin # Soluble lignin part of solvent stream 
        used_biomass.imass['l', 'SolubleLignin'] = 0 # No soluble lignin in biomass
        




    def _calculate_pressure_drop(self,
                                D,    
                                rho,                    
                                mu,                  
                                epsilon,              
                                u):
                                       
        Re = (D*rho*u)/mu
        if Re/(1-epsilon) < 500:
            dP = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
        elif 1000 < Re/(1-epsilon) < 5000:
            dP = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
        return dP
    
        

    def _design(self):
        diameter, length, actual_per_reactor_volume = self._size_bed()   # Calling size bed function to determine diameter and length 

        self.set_design_result('Diameter', 'ft', diameter)  
        self.set_design_result('Length', 'ft', length)
        self.set_design_result('Reactor volume', 'm3', actual_per_reactor_volume)


        
        # Calculates weight based off pressure, diameter and length
        # Adds vcessel type wall thickness, vessel weight, diameter and length to dictionary
        # But diameter and length are already there because of set_design_result above
        self.design_results.update(
            self._vertical_vessel_design(    
                self.P*(1/6894.76),
                self.design_results['Diameter'],
                self.design_results['Length']
            )
        )
        
                            

        pressure_drop = self._calculate_pressure_drop( 0.004,    # poplar particle diameter in [m]
        800, #self.ins[1].rho,                    # methanol water density [kg/m3]
        0.0006, # self.ins[1].get_property('mu', 'kg/m/s'), # methanol water viscosity [mu]
        0.5, #self.epsilon,     # Void fraction 
        0.1 # self.superficial_velocity * 3.28 # # superficial velocity [m/s]                                                
        )                  
        
        self.set_design_result('Pressure drop', 'bar', pressure_drop)
        self.pump_1.P = (self.P - self.ins[1].P) + pressure_drop
        self.pump_1.simulate()

        self.heat_exchanger_1.T = self.T
        self.heat_exchanger_1.simulate()
        





        

    def _cost(self):
        design = self.design_results # Calling the dictionary used to store design results in design method above 

        baseline_purchase_costs = self.baseline_purchase_costs # Dictionary for storing baseline costs

        weight = design['Weight']  # weight parameter stores the value from the 'Weight' key in the design dictionnary

        # Calculates the baseline purchase cost based off diameter length and weight
        baseline_purchase_costs.update( 
            self._vessel_purchase_cost(weight, design['Diameter'], design['Length'])
        )

        self.parallel['self'] = self.N_beds # Used to create multiple of the same beds
        # self.parallel['pump_1'] = 1


        
        '''
          **** Considerations needed *****
          - Pressure drop
          - Vessel material
          - How are 3 running and 1 regenerating? Need to add 
          - Void fraction and overdesign
          
          
        '''



        

        

In [ ]:
reactor = RCFReactor(ins = (poplar_in, meoh_in), N_beds=4, reaction = solvolysis)
reactor.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet Poplar_In from R3; Poplar_In is now docked at R4
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet Methanol_In from R3; Methanol_In is now docked at R4
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\hwadg\AppData\Local\Temp\ipykernel_46192\1590185241.py:202: CostWarning: <RCFReactor: R4> Vertical vessel length (75.6 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(


In [ ]:
reactor

RCFReactor: R4
ins...
[0] Poplar_In  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Extract    7.4
                        Acetate    48.6
                        Ash        1e+03
                        Lignin     156
                        Cellulose  238
                        Xylan      84.5
                        ...        27.5
[1] Methanol_In  
    phase: 'l', T: 300 K, P: 100000 Pa
    flow (kmol/hr): Water     5.53e+03
                    Methanol  2.2e+04
outs...
[0] s7  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Extract    7.4
                        Acetate    48.6
                        Ash        1e+03
                        Lignin     46.8
                        Cellulose  238
                        Xylan      84.5
                        ...       

In [ ]:
reactor.results()

RCFReactor                                                 Units       R12
Electricity         Power                                     kW  8.05e+03
                    Cost                                  USD/hr       630
High pressure steam Duty                                   kJ/hr  2.16e+09
                    Flow                                 kmol/hr  6.71e+04
                    Cost                                  USD/hr  2.13e+04
Design              Diameter                                  ft      12.9
                    Length                                    ft      75.6
                    Reactor volume                            m3  2.47e+03
                    Vessel type                                   Vertical
                    Weight                                    lb   8.7e+05
                    Wall thickness                            in       6.1
                    Pressure drop                            bar      6.74
Purchase cost       Vertical pressure vessel (x4)            USD  9.02e+06
                    Platform and ladders (x4)                USD  2.21e+05
                    Pump 1 - Pump (x4)                       USD  2.53e+06
                    Pump 1 - Motor (x24)                     USD  1.08e+06
                    Heat exchanger 1 - Floating head...      USD  1.47e+06
Total purchase cost                                          USD  1.43e+07
Utility cost                                              USD/hr  2.19e+04